Notes:

MAX_WORDS from 20000 to 10000 hurt accuracy and lead to overfitting with weighted embedding LSTM benchmark

In [2]:
import keras as k
import numpy as np
import pandas as pd
from keras.layers import GRU, Dense, Activation, Embedding, Input, Dropout, LSTM
from keras.layers import Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, one_hot
from keras.utils import np_utils

In [3]:
import tensorflow as tf

In [4]:
data = pd.read_csv('train.tsv',sep='\t')
test = pd.read_csv('test.tsv',sep='\t')

In [5]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


Preprocess data

In [7]:
#Create tokens and sequences with those tokens
MAX_WORDS = 20000

texts = data['Phrase'].values
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(texts)
tokenizer.fit_on_texts(test['Phrase'].values)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 17780 unique tokens.


In [8]:
#Pad the sequences to the max sequence length
MAX_SEQUENCE_LENGTH = max([len(i) for i in sequences])
x = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y = np_utils.to_categorical(data['Sentiment'].values)

In [7]:
#Create embedding matrix for the tokens
GRU_UNITS = 128
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,mask_zero=True))
model.add(GRU(GRU_UNITS))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
with tf.device('/gpu:0'):
    model.fit(x, y, batch_size=32, epochs=3, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 574s - loss: 0.9098 - acc: 0.6284 - val_loss: 0.9601 - val_acc: 0.6062
Epoch 2/3
124848/124848 [==============================] - 571s - loss: 0.7318 - acc: 0.6958 - val_loss: 0.9676 - val_acc: 0.6098
Epoch 3/3
124848/124848 [==============================] - 572s - loss: 0.6504 - acc: 0.7269 - val_loss: 0.9979 - val_acc: 0.6038


Thoughts:

Balance data set by removing a large portion of neutral reviews

Use pretrained encoding matrix 

In [ ]:
#Create embedding matrix for the tokens
GRU_UNITS = 256

model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,mask_zero=True))
model.add(LSTM(GRU_UNITS,recurrent_dropout=0.2,dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
with tf.device('/gpu:0'):
    model.fit(x, y, batch_size=32, epochs=3, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 714s - loss: 0.9383 - acc: 0.6219 - val_loss: 0.9704 - val_acc: 0.6070
Epoch 2/3
124848/124848 [==============================] - 752s - loss: 0.7478 - acc: 0.6926 - val_loss: 0.9734 - val_acc: 0.6100
Epoch 3/3
124848/124848 [==============================] - 978s - loss: 0.6654 - acc: 0.7218 - val_loss: 0.9874 - val_acc: 0.6089


In [ ]:
import gc
gc.collect()

In [9]:
embeddings_index = {}
f = open('glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [14]:
#Create embedding matrix for the tokens
GRU_UNITS = 256
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,mask_zero=True))
model.add(LSTM(GRU_UNITS,recurrent_dropout=0.2,dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
with tf.device('/gpu:0'):
    model.fit(x, y, batch_size=32, epochs=2, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 879s - loss: 0.9053 - acc: 0.6287 - val_loss: 0.9102 - val_acc: 0.6222
Epoch 2/3
124848/124848 [==============================] - 858s - loss: 0.7504 - acc: 0.6871 - val_loss: 0.9153 - val_acc: 0.6253
Epoch 3/3
124848/124848 [==============================] - 865s - loss: 0.6870 - acc: 0.7124 - val_loss: 0.9356 - val_acc: 0.6182


In [11]:
#Create embedding matrix for the tokens
GRU_UNITS = 256
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,mask_zero=True))
model.add(LSTM(GRU_UNITS,recurrent_dropout=0.2,dropout=0.2))

model.add(Dense(5,activation='softmax'))
model.add(Dropout(0.2))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x, y, batch_size=32, epochs=2, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/2


InternalError: Blas GEMM launch failed : a.shape=(1568, 100), b.shape=(100, 256), m=1568, n=256, k=100
	 [[Node: lstm_1/MatMul_3 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](lstm_1/Reshape_12, lstm_1/strided_slice_3)]]
	 [[Node: gradients/lstm_1/while/mul_grad/Reshape/_169 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2392_gradients/lstm_1/while/mul_grad/Reshape", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](^_cloopgradients/lstm_1/while/clip_by_value_2/Minimum_grad/zeros/Const/_11)]]

Caused by op 'lstm_1/MatMul_3', defined at:
  File "C:\Python\Python35\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Python\Python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Python\Python35\lib\site-packages\

In [21]:
#Create embedding matrix for the tokens
GRU_UNITS = 256
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,mask_zero=True,trainable=False))
model.add(LSTM(GRU_UNITS,recurrent_dropout=0.2,dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
with tf.device('/gpu:0'):
    model.fit(x, y, batch_size=32, epochs=10, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/10
124848/124848 [==============================] - 838s - loss: 0.9902 - acc: 0.5927 - val_loss: 0.9514 - val_acc: 0.6009
Epoch 2/10
124848/124848 [==============================] - 837s - loss: 0.9049 - acc: 0.6234 - val_loss: 0.9269 - val_acc: 0.6108
Epoch 3/10
124848/124848 [==============================] - 863s - loss: 0.8581 - acc: 0.6433 - val_loss: 0.9205 - val_acc: 0.6145
Epoch 4/10
124848/124848 [==============================] - 847s - loss: 0.8195 - acc: 0.6588 - val_loss: 0.9254 - val_acc: 0.6152
Epoch 5/10
124848/124848 [==============================] - 842s - loss: 0.7917 - acc: 0.6703 - val_loss: 0.9262 - val_acc: 0.6201
Epoch 6/10
124848/124848 [==============================] - 836s - loss: 0.7703 - acc: 0.6790 - val_loss: 0.9320 - val_acc: 0.6156
Epoch 7/10
124848/124848 [==============================] - 833s - loss: 0.7521 - acc: 0.6887 - val_loss: 0.9299 - val_acc: 0.6161
Epoch 8/10
124848/124848 [======

In [ ]:
#May be some training left to do here, try for more epochs

In [16]:
#Create embedding matrix for the tokens
GRU_UNITS = 256
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,mask_zero=True))
model.add(GRU(GRU_UNITS,recurrent_dropout=0.2,dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
with tf.device('/gpu:0'):
    model.fit(x, y, batch_size=32, epochs=3, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 789s - loss: 0.9163 - acc: 0.6217 - val_loss: 0.9118 - val_acc: 0.6237
Epoch 2/3
124848/124848 [==============================] - 677s - loss: 0.7678 - acc: 0.6798 - val_loss: 0.9140 - val_acc: 0.6215
Epoch 3/3
124848/124848 [==============================] - 814s - loss: 0.7086 - acc: 0.7051 - val_loss: 0.9279 - val_acc: 0.6192


In [9]:
#Try with less words:
MAX_WORDS = 20000
EMBEDDING_DIM = 50

texts = data['Phrase'].values
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(texts)
tokenizer.fit_on_texts(test['Phrase'].values)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

embeddings_index = {}
f = open('glove.6B.50d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
#Pad the sequences to the max sequence length
MAX_SEQUENCE_LENGTH = max([len(i) for i in sequences])
x = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y = np_utils.to_categorical(data['Sentiment'].values)

Found 17780 unique tokens.
Found 400000 word vectors.


In [11]:
#TODO try decay


In [20]:
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(256,3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(256,3,activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation = 'sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
with tf.device('/gpu:0'):
    model.fit(x, y, batch_size=32, epochs=10, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/10
124848/124848 [==============================] - 71s - loss: 0.9264 - acc: 0.5785 - val_loss: 0.9556 - val_acc: 0.5694
Epoch 2/10
124848/124848 [==============================] - 71s - loss: 0.8025 - acc: 0.6238 - val_loss: 0.9527 - val_acc: 0.5873
Epoch 3/10
124848/124848 [==============================] - 70s - loss: 0.7510 - acc: 0.6422 - val_loss: 1.0193 - val_acc: 0.5912
Epoch 4/10
124848/124848 [==============================] - 69s - loss: 0.7161 - acc: 0.6583 - val_loss: 0.9935 - val_acc: 0.5967
Epoch 5/10
124848/124848 [==============================] - 69s - loss: 0.6853 - acc: 0.6732 - val_loss: 0.9923 - val_acc: 0.5958
Epoch 6/10
124848/124848 [==============================] - 71s - loss: 0.6476 - acc: 0.7043 - val_loss: 1.0480 - val_acc: 0.5826
Epoch 7/10
124848/124848 [==============================] - 70s - loss: 0.6105 - acc: 0.7230 - val_loss: 1.0606 - val_acc: 0.5911
Epoch 8/10
124848/124848 [=============

In [14]:
# BEST MODEL
UNITS = 32
model = Sequential()
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM,weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,mask_zero=True))
model.add(LSTM(UNITS,recurrent_dropout=0.2,dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
with tf.device('/gpu:0'):
    model.fit(x, y, batch_size=64, epochs=3, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 328s - loss: 1.0457 - acc: 0.5819 - val_loss: 0.9545 - val_acc: 0.6059
Epoch 2/3
124848/124848 [==============================] - 327s - loss: 0.8602 - acc: 0.6426 - val_loss: 0.9319 - val_acc: 0.6168
Epoch 3/3
124848/124848 [==============================] - 320s - loss: 0.7963 - acc: 0.6684 - val_loss: 0.9343 - val_acc: 0.6195


In [15]:
with tf.device('/gpu:0'):
    model.fit(x, y, batch_size=64, epochs=3, verbose=1, validation_split=0.2)

Train on 124848 samples, validate on 31212 samples
Epoch 1/3
124848/124848 [==============================] - 328s - loss: 0.7549 - acc: 0.6860 - val_loss: 0.9428 - val_acc: 0.6136
Epoch 2/3
124848/124848 [==============================] - 330s - loss: 0.7290 - acc: 0.6958 - val_loss: 0.9563 - val_acc: 0.6087
Epoch 3/3
  6976/124848 [>.............................] - ETA: 278s - loss: 0.6862 - acc: 0.7160

KeyboardInterrupt: 

In [31]:
model.save('LSTM_Weights_Dropout.h5')

ImportError: `save_model` requires h5py.

In [33]:
#Test model, need csv output [PhraseId,Sentiment]:

sequences_test = tokenizer.texts_to_sequences(test['Phrase'].values)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
with tf.device('/gpu:0'):
    out = model.predict_classes(x_test,verbose=1)

66292/66292 [==============================] - 143s   


In [34]:
out

array([3, 3, 2, ..., 1, 1, 1], dtype=int64)

In [37]:
test.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,156061,8545,An intermittently pleasing but mostly routine ...,3
1,156062,8545,An intermittently pleasing but mostly routine ...,3
2,156063,8545,An,2
3,156064,8545,intermittently pleasing but mostly routine effort,3
4,156065,8545,intermittently pleasing but mostly routine,2


In [36]:
test['Sentiment'] = out

In [39]:
np.save('out',out)

In [41]:
submit = test[['PhraseId','Sentiment']]

In [44]:
submit.to_csv('out')